# Teste de validações do sistema BI

Este documento se trata dos testes/validações entre os dados apresentados no BI e os dados persistidos no banco de dados. Na cláusula where de cada validação é possível manipular os filtros para validar os filtros do BI.

In [ ]:
import psycopg2 as pg
import pandas as pd
from dotenv import load_dotenv 
import os

In [ ]:
load_dotenv()
conexao = pg.connect(database=os.getenv("database"),
                    user=os.getenv("user"),
                    password=os.getenv("password"),
                    host=os.getenv("host"),
                    port=os.getenv("port")
                     )
conectado = conexao.cursor()

### Transação por estado 

In [ ]:
query_grafico1='''SELECT a.uf, count(t.cod_transacao) from dw.contas co
inner join dw.transacoes t on t.cod_cliente = co.cod_cliente
inner join dw.agencia a on a.cod_agencia = co.cod_agencia
inner join dw.clientes cl on cl.cod_cliente = co.cod_cliente
where cl.nome='Amanda Martins' and a.nome='Agência Matriz'
and t.data_transacao between '2020/01/13' and '2023/01/15'
group by a.uf,a.cod_agencia order by 1 asc'''

In [ ]:
conectado.execute(query_grafico1)
resultado_grafico1 = conectado.fetchall()

In [ ]:
resultado_grafico1

### Distribuição do total de transação por tipo

In [ ]:
query_grafico2='''SELECT t.nome_transacao, sum(t.valor_transacao) from dw.contas co
inner join dw.transacoes t on t.cod_cliente = co.cod_cliente
inner join dw.agencia a on a.cod_agencia = co.cod_agencia
inner join dw.clientes cl on cl.cod_cliente = co.cod_cliente
where cl.nome='Amanda Martins' and a.nome='Agência Matriz'
AND t.data_transacao between '2016/01/17' and '2023/01/15'
group by t.nome_transacao order by 2 asc'''


In [ ]:
conectado.execute(query_grafico2)
resultado_grafico2 = conectado.fetchall()

In [ ]:
resultado_grafico2

### Ganho e ganhos por cliente

In [ ]:
query_grafico3_4='''with x as (
select p.*, (valor_prestacao * quantidade_parcelas)+valor_entrada as total 
from dw.credito p
where status_proposta='Aprovada'
--order by 2 asc
)

select ci.nome,sum(p.total) from dw.contas co
inner join x p on p.cod_cliente = co.cod_cliente
inner join dw.agencia a on a.cod_agencia = co.cod_agencia
inner join dw.clientes ci on ci.cod_cliente = co.cod_cliente
where ci.nome='André Ferreira' and a.nome='Agência Campinas'
group by ci.nome order by 1 asc'''

In [ ]:
conectado.execute(query_grafico3_4)
resultado_grafico3_4 = conectado.fetchall()

In [ ]:
resultado_grafico3_4

### Transação positiva e negativa por tempo 

In [ ]:
#TRANSAÇÕES POSITIVAS
query_grafico5='''with x as(
select t.*, cast(data_transacao as date) as dt_transf from dw.transacoes t
where valor_transacao > 0)

select ci.cod_cliente,dt_transf,sum(valor_transacao) as qtd from x
inner join dw.clientes ci on x.cod_cliente = ci.cod_cliente
inner join dw.contas co on co.cod_cliente = x.cod_cliente
INNER JOIN dw.agencia a on a.cod_agencia = co.cod_agencia
where ci.nome='André Ferreira' and a.nome='Agência Campinas'
AND x.data_transacao between '2010/02/27' and '2023/01/15'
group by ci.cod_cliente,dt_transf order by 2 asc'''

In [ ]:
conectado.execute(query_grafico5)
resultado_grafico5 = conectado.fetchall()

In [ ]:
resultado_grafico5

In [ ]:
#TRANSAÇÕES NEGATIVAS
query_grafico5='''with x as(
select t.*, cast(data_transacao as date) as dt_transf from dw.transacoes t
where valor_transacao < 0)

select ci.cod_cliente,dt_transf,sum(valor_transacao) as qtd from x
inner join dw.clientes ci on x.cod_cliente = ci.cod_cliente
inner join dw.contas co on co.cod_cliente = x.cod_cliente
INNER JOIN dw.agencia a on a.cod_agencia = co.cod_agencia
where ci.nome='André Ferreira' and a.nome='Agência Campinas'
AND x.data_transacao between '2010/02/27' and '2023/01/15'
group by ci.cod_cliente,dt_transf order by 2 asc'''

In [ ]:
conectado.execute(query_grafico5)
resultado_grafico5 = conectado.fetchall()

In [ ]:
resultado_grafico5